In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Cargar variables de entorno
load_dotenv()

# Inicializar cliente OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Configuración básica del agente
agent_config = {
    "model": "gpt-4",
    "temperature": 0.7,
    "max_tokens": 1000,
    "verbose": True,
    "task_decomposition_depth": 2  # Nivel de descomposición de tareas
}

print("✅ Conexión a API configurada")

✅ Conexión a API configurada


In [2]:
def descomponer_objetivo(objetivo, contexto=None):
    """
    Función que descompone un objetivo en tareas específicas usando OpenAI
    """
    prompt = f"""
    Actúa como un asistente experto en planificación. Te daré un objetivo, y tu tarea es:
    
    1. Descomponer el objetivo en máximo 5 sub-tareas específicas y accionables
    2. Cada tarea debe estar numerada y ser clara
    3. Las tareas deben estar en orden lógico de ejecución
    4. Cada tarea debe tener un criterio de éxito claro
    
    OBJETIVO: {objetivo}
    
    FORMATO DE RESPUESTA:
    {{
        "tareas": [
            {{
                "id": 1,
                "descripcion": "Descripción de la tarea 1",
                "criterio_exito": "Cómo saber que se completó exitosamente"
            }},
            ...
        ]
    }}
    
    Responde solo con el JSON, sin texto adicional.
    """
    
    # Llamada a la API
    response = client.chat.completions.create(
        model=agent_config["model"],
        messages=[{"role": "user", "content": prompt}],
        temperature=agent_config["temperature"],
        max_tokens=agent_config["max_tokens"]
    )
    
    # Extraer y retornar el resultado
    return response.choices[0].message.content

# Probar con un objetivo simple
objetivo_prueba = "Escribe un correo de invitación para una conferencia sobre IA"
tareas_json = descomponer_objetivo(objetivo_prueba)
print(tareas_json)

{
    "tareas": [
        {
            "id": 1,
            "descripcion": "Investiga la conferencia sobre IA, incluyendo los temas que se discutirán, los oradores principales y la fecha y hora del evento.",
            "criterio_exito": "Tienes suficiente información sobre la conferencia para escribir una descripción precisa en el correo de invitación."
        },
        {
            "id": 2,
            "descripcion": "Identificar al público objetivo del correo electrónico, como profesionales de la IA, estudiantes, etc.",
            "criterio_exito": "Has creado una lista de destinatarios para el correo de invitación a la conferencia."
        },
        {
            "id": 3,
            "descripcion": "Escribir el borrador del correo de invitación que incluye una descripción atractiva de la conferencia, los detalles del evento y una llamada a la acción para que los destinatarios se registren o respondan al correo.",
            "criterio_exito": "Has escrito un borrador del cor

In [3]:
import json
from datetime import datetime

def ejecutar_tarea(tarea, contexto=None):
    """
    Ejecuta una tarea individual usando la API de OpenAI
    """
    # Construir el prompt para la ejecución de la tarea
    prompt = f"""
    Actúa como un asistente profesional ejecutando una tarea específica.
    
    TAREA: {tarea['descripcion']}
    CRITERIO DE ÉXITO: {tarea['criterio_exito']}
    
    Por favor, ejecuta esta tarea con la máxima calidad posible. 
    Entrega el resultado directo, sin explicaciones adicionales.
    """
    
    # Llamada a la API
    response = client.chat.completions.create(
        model=agent_config["model"],
        messages=[{"role": "user", "content": prompt}],
        temperature=agent_config["temperature"],
        max_tokens=agent_config["max_tokens"]
    )
    
    # Extraer resultado
    resultado = response.choices[0].message.content
    
    # Registrar la ejecución
    print(f"✓ Tarea {tarea['id']} completada: {tarea['descripcion'][:50]}...")
    
    return resultado

# Probar ejecutando la primera tarea
try:
    # Parsear las tareas desde el JSON obtenido anteriormente
    tareas = json.loads(tareas_json)['tareas']
    
    # Ejecutar la primera tarea
    resultado_tarea = ejecutar_tarea(tareas[0])
    print("\nRESULTADO:")
    print(resultado_tarea)
except json.JSONDecodeError:
    print("Error: El formato JSON no es válido. Revisando la respuesta original:")
    print(tareas_json)

✓ Tarea 1 completada: Investiga la conferencia sobre IA, incluyendo los ...

RESULTADO:
Después de realizar la investigación, encontré la siguiente información sobre la conferencia de IA:

Nombre del evento: Global Artificial Intelligence Conference 2022

Fecha y hora: Del 15 al 17 de septiembre de 2022, desde las 9:00 a.m. hasta las 5:00 p.m. cada día.

Lugar: Centro de Convenciones de Silicon Valley, California, EE. UU.

Temas clave de discusión:

1. Inteligencia Artificial (IA) y el Futuro del Trabajo
2. Avances en Aprendizaje Automático y Aprendizaje Profundo
3. IA en el Sector de la Salud
4. IA en el Comercio Electrónico
5. Ética y Responsabilidad en IA
6. IA y Big Data

Oradores principales:

1. Dr. John Smith, Vicepresidente de IA en IBM
2. Dra. Jane Doe, Directora de IA en Google
3. Dr. Albert Wong, Científico Principal en Adobe Systems
4. Dra. Michelle Brown, Profesora de IA en la Universidad de Stanford
5. Sr. Michael Johnson, CEO de AI Tech

Por favor, hágamelo saber si nece

In [4]:
def ejecutar_objetivo(objetivo):
    """
    Función principal que ejecuta el flujo completo del agente:
    1. Descompone el objetivo en tareas
    2. Ejecuta cada tarea secuencialmente
    3. Integra los resultados
    """
    print(f"🎯 Iniciando objetivo: {objetivo}")
    print("=" * 50)
    
    # Paso 1: Descomponer el objetivo en tareas
    print("Descomponiendo objetivo en tareas...")
    tareas_json = descomponer_objetivo(objetivo)
    
    try:
        tareas = json.loads(tareas_json)['tareas']
    except (json.JSONDecodeError, KeyError):
        print("Error al parsear las tareas. Usando formato alternativo...")
        # Intento de recuperación básico
        tareas = [{"id": 1, "descripcion": objetivo, "criterio_exito": "Completar el objetivo"}]
    
    print(f"✓ Se identificaron {len(tareas)} tareas")
    print("-" * 50)
    
    # Paso 2: Ejecutar cada tarea secuencialmente
    resultados = []
    for tarea in tareas:
        print(f"\n📌 Ejecutando tarea {tarea['id']}: {tarea['descripcion']}")
        resultado = ejecutar_tarea(tarea)
        resultados.append(resultado)
        print("-" * 30)
    
    # Paso 3: Integrar resultados si hay múltiples tareas
    if len(resultados) > 1:
        print("\n🔄 Integrando resultados...")
        integracion_prompt = f"""
        Actúa como un asistente experto en síntesis de información.
        
        Has completado las siguientes tareas relacionadas con el objetivo: "{objetivo}"
        
        RESULTADOS:
        {' '.join([f'Tarea {i+1}: {r[:200]}...' for i, r in enumerate(resultados)])}
        
        Por favor, integra estos resultados en un producto final coherente y completo.
        """
        
        response = client.chat.completions.create(
            model=agent_config["model"],
            messages=[{"role": "user", "content": integracion_prompt}],
            temperature=agent_config["temperature"],
            max_tokens=agent_config["max_tokens"] * 2
        )
        
        resultado_final = response.choices[0].message.content
    else:
        resultado_final = resultados[0]
    
    print("\n✅ OBJETIVO COMPLETADO")
    print("=" * 50)
    print("\nRESULTADO FINAL:")
    print(resultado_final)
    
    return resultado_final

# Ejecutar con el objetivo de prueba
objetivo_prueba = "Escribe un correo de invitación para una conferencia sobre IA que se realizará el próximo mes"
ejecutar_objetivo(objetivo_prueba)

🎯 Iniciando objetivo: Escribe un correo de invitación para una conferencia sobre IA que se realizará el próximo mes
Descomponiendo objetivo en tareas...
✓ Se identificaron 5 tareas
--------------------------------------------------

📌 Ejecutando tarea 1: Investigar los detalles clave de la conferencia
✓ Tarea 1 completada: Investigar los detalles clave de la conferencia...
------------------------------

📌 Ejecutando tarea 2: Identificar y recopilar los datos de contacto de los destinatarios
✓ Tarea 2 completada: Identificar y recopilar los datos de contacto de l...
------------------------------

📌 Ejecutando tarea 3: Redactar el cuerpo del correo
✓ Tarea 3 completada: Redactar el cuerpo del correo...
------------------------------

📌 Ejecutando tarea 4: Revisar y corregir el borrador del correo
✓ Tarea 4 completada: Revisar y corregir el borrador del correo...
------------------------------

📌 Ejecutando tarea 5: Enviar el correo a los destinatarios
✓ Tarea 5 completada: Enviar el co

'Estimado [Nombre del destinatario],\n\nEspero que este mensaje te encuentre bien. Me gustaría extender una invitación personal para que asistas a nuestra próxima conferencia titulada "Inteligencia Artificial y su Futuro". Esta conferencia es una parte de nuestra serie Anual de Innovación Tecnológica y contará con una variedad de oradores destacados en el campo de la inteligencia artificial.\n\nLa conferencia se llevará a cabo el 25 de Mayo de 2023, de 9:00 AM a 5:00 PM (Hora local), en el Centro de Convenciones Internacionales, Nueva York, NY. También hemos previsto la opción de asistencia virtual para aquellos que no puedan estar presentes físicamente.\n\nAdemás, me gustaría informarte que nuestra reunión de planificación estratégica se llevará a cabo el próximo lunes. Te agradecería que confirmaras tu asistencia a esta reunión y a la conferencia.\n\nLamento informar que, como asistente AI, no tengo la capacidad de enviar este correo directamente. Sin embargo, puedes copiar el texto 